Wendy's Project  
I am creating a predictive ranking of NFL players based on their stats across an entire season  
My impetus for this project is that my fantasy football team sucks  
I am going to focus on offensive players only  
I am going to focus on player positions in my roster only  
This means QB, WR, RB, TE, K

In [1]:
# Bonus, I could try to see if I could predict improvement btwn 2010 and 2016
# not sure how but something worth thinking about

In [3]:
# read the data into a DataFrame named nfl16
import pandas as pd
path = '../GA_Class_Project/'
url = path + 'season2016_wy.csv'
nfl16 = pd.read_csv(url, index_col = 0)
nfl16.head()

,id,home,team,pos,defense_ast,defense_ffum,defense_int,defense_sk,defense_tkl,fumbles_lost,...,receiving_twopta,receiving_twoptm,receiving_yds,rushing_att,rushing_lng,rushing_lngtd,rushing_tds,rushing_twopta,rushing_twoptm,rushing_yds
name,,,,,,,,,,,,,,,,,,,,,
T.Siemian,00-0032156,no,DEN,QB,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,28.0,42.0,0.0,0.0,1.0,0.0,57.0
A.Janovich,00-0032956,no,DEN,FB,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,44.0,4.0,33.0,28.0,1.0,0.0,0.0,33.0
D.Booker,00-0032972,no,DEN,RB,NaN,NaN,NaN,NaN,NaN,3.0,...,0.0,0.0,265.0,174.0,152.0,21.0,4.0,0.0,0.0,612.0
C.Anderson,00-0029854,no,DEN,RB,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,128.0,110.0,106.0,13.0,4.0,0.0,0.0,437.0
V.Green,00-0028142,no,DEN,TE,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,237.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# what are we looking at?
nfl16.shape
# 1573 players in this list with 59 attributes

(1573, 59)

In [5]:
# let's just look at offensive players
# this includes QB, RB, WR, TE, K
nfl16.pos.value_counts()

WR     205
RB     146
DE     144
CB     129
TE     115
OLB     93
DT      90
DB      79
QB      71
LB      66
SS      52
ILB     44
FS      43
K       35
P       35
NT      25
OT      19
FB      18
MLB     16
C       16
OG      12
T        7
G        7
LS       6
SAF      3
Name: pos, dtype: int64

In [6]:
# create new DataFrames using only QB, WR, RB, TE, K
nfl16_QB = nfl16.loc[nfl16.pos == 'QB']
nfl16_WR = nfl16.loc[nfl16.pos == 'WR']
nfl16_RB = nfl16.loc[nfl16.pos == 'RB']
nfl16_TE = nfl16.loc[nfl16.pos == 'TE']
nfl16_K = nfl16.loc[nfl16.pos == 'K']

In [7]:
# concat all of those and call them offense
frames = [nfl16_QB, nfl16_WR, nfl16_RB, nfl16_TE, nfl16_K]
nfl16_off = pd.concat(frames)
nfl16_off.shape

(572, 59)

In [8]:
nfl16_off.head()

,id,home,team,pos,defense_ast,defense_ffum,defense_int,defense_sk,defense_tkl,fumbles_lost,...,receiving_twopta,receiving_twoptm,receiving_yds,rushing_att,rushing_lng,rushing_lngtd,rushing_tds,rushing_twopta,rushing_twoptm,rushing_yds
name,,,,,,,,,,,,,,,,,,,,,
T.Siemian,00-0032156,no,DEN,QB,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,28.0,42.0,0.0,0.0,1.0,0.0,57.0
C.Newton,00-0027939,no,CAR,QB,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,90.0,162.0,14.0,5.0,1.0,1.0,359.0
M.Ryan,00-0026143,no,ATL,QB,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,35.0,94.0,0.0,0.0,1.0,0.0,117.0
J.Winston,00-0031503,no,TB,QB,NaN,NaN,NaN,NaN,NaN,6.0,...,NaN,NaN,NaN,52.0,121.0,7.0,1.0,1.0,0.0,168.0
J.Flacco,00-0026158,no,BAL,QB,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,22.0,55.0,8.0,2.0,0.0,0.0,53.0


In [9]:
nfl16_off.columns.values.tolist()

['id',
 'home',
 'team',
 'pos',
 'defense_ast',
 'defense_ffum',
 'defense_int',
 'defense_sk',
 'defense_tkl',
 'fumbles_lost',
 'fumbles_rcv',
 'fumbles_tot',
 'fumbles_trcv',
 'fumbles_yds',
 'kicking_fga',
 'kicking_fgm',
 'kicking_fgyds',
 'kicking_totpfg',
 'kicking_xpa',
 'kicking_xpb',
 'kicking_xpmade',
 'kicking_xpmissed',
 'kicking_xptot',
 'kickret_avg',
 'kickret_lng',
 'kickret_lngtd',
 'kickret_ret',
 'kickret_tds',
 'passing_att',
 'passing_cmp',
 'passing_ints',
 'passing_tds',
 'passing_twopta',
 'passing_twoptm',
 'passing_yds',
 'punting_avg',
 'punting_i20',
 'punting_lng',
 'punting_pts',
 'punting_yds',
 'puntret_avg',
 'puntret_lng',
 'puntret_lngtd',
 'puntret_ret',
 'puntret_tds',
 'receiving_lng',
 'receiving_lngtd',
 'receiving_rec',
 'receiving_tds',
 'receiving_twopta',
 'receiving_twoptm',
 'receiving_yds',
 'rushing_att',
 'rushing_lng',
 'rushing_lngtd',
 'rushing_tds',
 'rushing_twopta',
 'rushing_twoptm',
 'rushing_yds']

In [10]:
#create dataframe with only offensive stats
"""nfl16_off = nfl16_off[['id', 'home', 'team', 'pos', 'fumbles_lost', 'fumbles_rcv', 'fumbles_tot', 'fumbles_trcv',
 'fumbles_yds', 'kicking_fga', 'kicking_fgm', 'kicking_fgyds', 'kicking_totpfg', 'kicking_xpa', 'kicking_xpb',
 'kicking_xpmade', 'kicking_xpmissed', 'kicking_xptot', 'kickret_avg', 'kickret_lng', 'kickret_lngtd',
 'kickret_ret', 'kickret_tds', 'passing_att', 'passing_cmp', 'passing_ints', 'passing_tds', 'passing_twopta',
 'passing_twoptm', 'passing_yds', 'receiving_lng', 'receiving_lngtd', 'receiving_rec', 'receiving_tds',
 'receiving_twopta', 'receiving_twoptm', 'receiving_yds', 'rushing_att', 'rushing_lng', 'rushing_lngtd',
 'rushing_tds', 'rushing_twopta', 'rushing_twoptm', 'rushing_yds']]"""

"nfl16_off = nfl16_off[['id', 'home', 'team', 'pos', 'fumbles_lost', 'fumbles_rcv', 'fumbles_tot', 'fumbles_trcv',\n 'fumbles_yds', 'kicking_fga', 'kicking_fgm', 'kicking_fgyds', 'kicking_totpfg', 'kicking_xpa', 'kicking_xpb',\n 'kicking_xpmade', 'kicking_xpmissed', 'kicking_xptot', 'kickret_avg', 'kickret_lng', 'kickret_lngtd',\n 'kickret_ret', 'kickret_tds', 'passing_att', 'passing_cmp', 'passing_ints', 'passing_tds', 'passing_twopta',\n 'passing_twoptm', 'passing_yds', 'receiving_lng', 'receiving_lngtd', 'receiving_rec', 'receiving_tds',\n 'receiving_twopta', 'receiving_twoptm', 'receiving_yds', 'rushing_att', 'rushing_lng', 'rushing_lngtd',\n 'rushing_tds', 'rushing_twopta', 'rushing_twoptm', 'rushing_yds']]"

In [11]:
#nfl16_off.shape

### Points earned in fantasy, ignoring PPR   
#### Offensive Players  
Passing Yards: 1 point per 25 yards  
Passing Touchdowns: 4 points  
Passing Interceptions: -2 points  
Rushing Yards: 1 point per 10 yards  
Rushing Touchdowns: 6 points  
Receptions: 1 points (only if using PPR scoring)  
Receiving Yards: 1 point per 10 yards  
Receiving Touchdowns: 6 points  
2-Point Conversions: 2 points  
Fumbles Lost: -2 points  
Fumble Recovered for a Touchdown: 6 points  

#### Kicking  
PAT Made: 1 point  
FG Made (0-49 yards): 3 points  
FG Made (50+ yards): 5 points
 

In [12]:
nfl16_off.head()

,id,home,team,pos,defense_ast,defense_ffum,defense_int,defense_sk,defense_tkl,fumbles_lost,...,receiving_twopta,receiving_twoptm,receiving_yds,rushing_att,rushing_lng,rushing_lngtd,rushing_tds,rushing_twopta,rushing_twoptm,rushing_yds
name,,,,,,,,,,,,,,,,,,,,,
T.Siemian,00-0032156,no,DEN,QB,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,28.0,42.0,0.0,0.0,1.0,0.0,57.0
C.Newton,00-0027939,no,CAR,QB,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,90.0,162.0,14.0,5.0,1.0,1.0,359.0
M.Ryan,00-0026143,no,ATL,QB,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,35.0,94.0,0.0,0.0,1.0,0.0,117.0
J.Winston,00-0031503,no,TB,QB,NaN,NaN,NaN,NaN,NaN,6.0,...,NaN,NaN,NaN,52.0,121.0,7.0,1.0,1.0,0.0,168.0
J.Flacco,00-0026158,no,BAL,QB,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,22.0,55.0,8.0,2.0,0.0,0.0,53.0


In [36]:
#map stats to points using list comprehensions
#is there a better way to do this?
fumbles_lost_pts = [x * -2 for x in nfl16_off.fumbles_lost]
passing_yds_pts = [x / 25 for x in nfl16_off.passing_yds]
passing_tds_pts = [x * 4 for x in nfl16_off.passing_tds]
passing_ints_pts = [x * -2 for x in nfl16_off.passing_ints]
rushing_yds_pts = [x /10 for x in nfl16_off.rushing_yds]
rushing_tds_pts = [x * 6 for x in nfl16_off.rushing_tds]
receiving_yds_pts = [x / 10 for x in nfl16_off.receiving_yds]
receiving_tds_pts = [x * 6 for x in nfl16_off.receiving_tds]
passing_twoptm_pts = [x * 2 for x in nfl16_off.passing_twoptm]
receiving_twoptm_pts = [x * 2 for x in nfl16_off.receiving_twoptm]
rushing_twoptm_pts = [x * 2 for x in nfl16_off.rushing_twoptm]
kicking_xpmade_pts = [x * 1 for x in nfl16_off.kicking_xpmade]


In [40]:
# fgyds doesn't work across entire season
# maybe find a loop to calculate this separately?
# I think rushing/receiving yards still okay bc we're not counting any bonuses per game

kicking_fgyds_pts = [x * 3 if x < 49 else x * 5 in nfl16_off.kicking_fgyds]

In [41]:
kicking_fgyds_pts

[33.0]

In [42]:
nfl16_off.kicking_fgyds

name
T.Siemian             NaN
C.Newton              NaN
M.Ryan                NaN
J.Winston             NaN
J.Flacco              NaN
T.Taylor              NaN
B.Osweiler            NaN
J.Cutler              NaN
B.Bortles             NaN
A.Rodgers             NaN
A.Smith               NaN
P.Rivers              NaN
D.Brees               NaN
D.Carr                NaN
R.Fitzpatrick         NaN
A.Dalton              NaN
C.Wentz               NaN
M.Mariota             NaN
R.Wilson              NaN
R.Tannehill           NaN
D.Prescott            NaN
E.Manning             NaN
R.Griffin             NaN
A.Luck                NaN
M.Stafford            NaN
C.Palmer              NaN
J.Garoppolo           NaN
K.Cousins             NaN
B.Roethlisberger      NaN
B.Gabbert             NaN
                    ...  
D.Carpenter         451.0
N.Novak             560.0
C.Barth             445.0
J.Myers             583.0
M.Crosby            541.0
C.Santos            523.0
J.Lambo             484.0
W.Lutz 